In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [1]:
from platform import python_version
print(python_version())

3.10.12


In [46]:
!pip install torch


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [47]:
!pip install torchvision transformers pandas scikit-learn
!pip install tqdm



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [48]:
import torch

In [68]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report
import re

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [69]:
print(device)

cuda


In [70]:
df = pd.read_csv("redacted_data_unique_cleaned_filtered.csv",encoding="latin_1",  names=['target', 'ids', 'date', 'flag', 'user', 'text'],header=1)

In [71]:
#df = df2.head(10)
#df.head(5)
# Assuming df is your DataFrame with the target column named 'target_column'
df_target_0 = df[df['target'] == 0].head(1000)
df_target_1 = df[df['target'] == 4].head(1000)

# If you want to combine both dataframes into one:
df = pd.concat([df_target_0, df_target_1])



In [72]:
df['target'].value_counts()

target
0    1000
4    1000
Name: count, dtype: int64

In [73]:
df["text"][0]

'[REDACTED] que me muera ? '

In [74]:
def clean_text(text):
    # Remove mentions
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = re.sub(' +', ' ', text)
    # Remove leading/trailing whitespace
    text = text.strip()
    return text

In [75]:
# Tokenize the text and convert them to tensors
def tokenize_text(text_list, tokenizer, max_length):
    input_ids = []
    attention_masks = []
    #print(len(text_list))
    count = 0
    for text in text_list:

        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_length,
                            pad_to_max_length=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        count+=1

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

In [76]:
# Apply cleaning function to the 'text' column
#df['text'] = df['text'].apply(clean_text)
# Clean text (You may add more cleaning steps as needed)
df['text'] = df['text'].apply(lambda x: x.lower())  # Convert text to lowercase
# Map label 4 to 1 for positive sentiment
df['target'] = df['target'].map({0: 0, 4: 1})

In [77]:
df["text"][0]
df['target'].value_counts()

target
0    1000
1    1000
Name: count, dtype: int64

In [78]:
# Split dataset into train, validation, and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df['target'], random_state=42, test_size=0.3, stratify=df['target'])
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, random_state=42, test_size=0.5, stratify=temp_labels)

In [79]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [80]:
max_len = 128  # Maximum length of input sequence
train_inputs, train_masks = tokenize_text(train_text.tolist(), tokenizer, max_len)
val_inputs, val_masks = tokenize_text(val_text.tolist(), tokenizer, max_len)
test_inputs, test_masks = tokenize_text(test_text.tolist(), tokenizer, max_len)

# Convert labels to tensors
train_labels = torch.tensor(train_labels.tolist())
val_labels = torch.tensor(val_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())

# Create data loaders
batch_size = 128
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [81]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [82]:
import tqdm

In [83]:
# Training loop
import time
epoch_time = time.time()
step_time = time.time()
print(f"Total Processing Time: {time.time() - epoch_time:.3f} seconds")
epoch_time = time.time()
step_time = time.time()
for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        if step % 100 == 0:
            print(step)
            print(f"Total Step Time: {time.time() - step_time:.3f} seconds")
            step_time = time.time()
    print(f"Total Epoch Time: {time.time() - epoch_time:.3f} seconds")
    epoch_time = time.time()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print("Average training loss: {0:.2f}".format(avg_train_loss))

# Validation loop
model.eval()
val_preds, val_true_labels = [], []
for batch in val_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to('cpu').numpy()

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    val_preds.extend(logits.argmax(axis=1))
    val_true_labels.extend(b_labels)

val_accuracy = accuracy_score(val_true_labels, val_preds)
print("Validation Accuracy: {0:.2f}".format(val_accuracy))

# Test loop
test_preds, test_true_labels = [], []
for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to('cpu').numpy()

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    test_preds.extend(logits.argmax(axis=1))
    test_true_labels.extend(b_labels)

test_accuracy = accuracy_score(test_true_labels, test_preds)
print("Test Accuracy: {0:.2f}".format(test_accuracy))

# Print classification report
print(classification_report(test_true_labels, test_preds))


Total Processing Time: 0.000 seconds
0
Total Step Time: 0.271 seconds
Total Epoch Time: 2.964 seconds
Average training loss: 0.68
0
Total Step Time: 2.964 seconds
Total Epoch Time: 2.965 seconds
Average training loss: 0.63
0
Total Step Time: 2.965 seconds
Total Epoch Time: 2.970 seconds
Average training loss: 0.59
Validation Accuracy: 0.71
Test Accuracy: 0.64
              precision    recall  f1-score   support

           0       0.65      0.61      0.63       150
           1       0.64      0.67      0.65       150

    accuracy                           0.64       300
   macro avg       0.64      0.64      0.64       300
weighted avg       0.64      0.64      0.64       300



In [51]:
model_path = "redacted_data_fine_tuned_model_10000_filtered"
torch.save(model, model_path)

Code to check accuracy of trained model

In [84]:

from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd

df = pd.read_csv("redacted_data_unique_cleaned_filtered.csv",encoding="latin_1",  names=['target', 'ids', 'date', 'flag', 'user', 'text'],header=1)
df_target_0 = df[df['target'] == 0].sample(n=1000)
df_target_1 = df[df['target'] == 4].sample(n=1000)

# If you want to combine both dataframes into one:
df = pd.concat([df_target_0, df_target_1])

# Load the saved model
model_path = "v"
model = torch.load(model_path)
model.eval()

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load the DataFrame
# Assuming 'df' is your DataFrame
df['target'] = df['target'].map({0: 0, 4: 1})  # Map target values to binary

# Tokenize the text column
encoded_inputs = tokenizer(list(df['text']), padding=True, truncation=True, max_length=128, return_tensors='pt')

# Ensure that the model and input tensors are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
encoded_inputs.to(device)

# Perform inference
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Get the predicted labels
predicted_labels = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Calculate accuracy
accuracy = (predicted_labels == df['target'].values).mean()
print(f"Accuracy: {accuracy}")


FileNotFoundError: [Errno 2] No such file or directory: 'v'

In [118]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
import torch
import pandas as pd

# Load the saved model
model_path = "faked_data_fine_tuned_model_100000"
model = torch.load(model_path)
model.eval()

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load the DataFrame
df = pd.read_csv("faked_data_unique.csv", encoding="latin_1", names=['target', 'ids', 'date', 'flag', 'user', 'text'], header=1)
df_target_0 = df[df['target'] == 0].sample(n=1000)
df_target_1 = df[df['target'] == 4].sample(n=1000)
df = pd.concat([df_target_0, df_target_1])
df['target'] = df['target'].map({0: 0, 4: 1})

# Define a custom Dataset class
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, texts, targets, max_length=128):
        self.tokenizer = tokenizer
        self.texts = texts
        self.targets = targets
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        target = self.targets[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'targets': torch.tensor(target)}

# Create an instance of the custom Dataset class
dataset = TextDataset(tokenizer, df['text'].tolist(), df['target'].tolist())

# Define the DataLoader
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size)

# Ensure that the model and input tensors are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Perform inference
model.eval()
all_predictions = []
all_targets = []
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predictions.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())


all_predictions_int = np.array(all_predictions)
all_targets_int = np.array(all_targets)

# Calculate accuracy
accuracy = (all_predictions_int == all_targets_int).mean()
print(f"Accuracy: {accuracy}")


Accuracy: 0.8515


In [112]:
import numpy as np

# Convert boolean array to integers
all_predictions_int = np.array(all_predictions)
all_targets_int = np.array(all_targets)

# Calculate accuracy
accuracy = (all_predictions_int == all_targets_int).mean()
print(f"Accuracy: {accuracy}")


Accuracy: 0.895


In [85]:
df['target'].value_counts()

target
0    1000
4    1000
Name: count, dtype: int64